In [13]:
import { zodToJsonSchema } from "zod-to-json-schema";
import { z } from "zod";
const getCurrentWeatherSchema = z.object({
    location: z.string().describe("The city and state, e.g. San Francisco, CA"),
    unit: z.enum(["celsius", "fahrenheit"]).describe("The unit of temperature"),
})

const paramSchema = zodToJsonSchema(getCurrentWeatherSchema)
paramSchema

{
  type: "object",
  properties: {
    location: {
      type: "string",
      description: "The city and state, e.g. San Francisco, CA"
    },
    unit: {
      type: "string",
      enum: [ "celsius", "fahrenheit" ],
      description: "The unit of temperature"
    }
  },
  required: [ "location", "unit" ],
  additionalProperties: false,
  "$schema": "http://json-schema.org/draft-07/schema#"
}

In [ ]:
// import { Ollama } from "@langchain/community/llms/ollama";
import { ChatOpenAI } from "@langchain/openai"
const model = new ChatOpenAI({
    temperature: 0 
})

const modelWithTools = model.bind({
    tools: [
        {
            type: "function",
            function: {
                name: "getCurrentWeather",
                description: "Get the current weather in a given location",
                parameters: zodToJsonSchema(getCurrentWeatherSchema),
            }
        }
    ]
})

await modelWithTools.invoke("北京的天气怎么样");

SyntaxError: Invalid shorthand property initializer

In [18]:
import { ChatPromptTemplate } from "@langchain/core/prompts";

const prompt = ChatPromptTemplate.fromMessages([
    ["system", "You are a helpful assistant"],
    ["human", "{input}"]
])

const chain = prompt.pipe(modelWithTools)

await chain.invoke({
    input: "北京的天气怎么样"
})

Error: Connection error.

In [19]:
const getCurrentTimeSchema = z.object({
    format: z
        .enum(["iso", "local", "string"])
        .optional()
        .describe("The format of the time, e.g. iso, locale, string")
})
zodToJsonSchema(getCurrentTimeSchema)

{
  type: "object",
  properties: {
    format: {
      type: "string",
      enum: [ "iso", "local", "string" ],
      description: "The format of the time, e.g. iso, locale, string"
    }
  },
  additionalProperties: false,
  "$schema": "http://json-schema.org/draft-07/schema#"
}

In [20]:
import { JsonOutputToolsParser } from "@langchain/core/output_parsers/openai_tools"
const taggingSchema = z.object({
    emotion: z.enum(["pos", "neg", "neutral"]).describe("文本的情感"),
    language: z.string().describe("文本的核心语言（应为ISO 639-1代码）")
})
const model = new ChatOpenAI({
    temperature: 0 
})

const modelTagging = model.bind({
    tools: [
        {
            type: "function",
            function: {
                name: "tagging",
                description: "为特定的文本片段打上标签",
                parameters: zodToJsonSchema(taggingSchema)
            }
        }
    ],
    tool_choice: {
        type: "function",
        function: {
           name: "tagging"
        }
    }
})

const prompt = ChatPromptTemplate.fromMessages([
    ["system", "仔细思考， 你有充足的时间进行严谨的思考， 然后按照指示对文本进行标记"],
    ["human", "{input}"]
])
const chain = prompt.pipe(modelTagging).pipe(new JsonOutputToolsParser())
await chain.invoke({
    input: "hello world"
})

Error: Connection error.